In [1]:
import numpy as np
from astropy.stats import mad_std
from photutils.detection import DAOStarFinder
from scipy.optimize import curve_fit

#conda install numba & conda install cudatoolkit

@jit(target_backend='cuda') 
def fwhm_new(image, fwhm=3.0, threshold_multiplier=5.0, size=10):
    image_data = image
    background_noise = mad_std(image_data[1])
    first_layer = image_data.reshape(512, 512)
    daofind = DAOStarFinder(fwhm=fwhm, threshold=threshold_multiplier * background_noise)
    sources = daofind(first_layer)

    if sources is None:
        print("Yıldızlar bulunamadı....:(")
        print(1)
        return None

    try:
        sorted_sources = sorted(sources, key=lambda x: x['flux'], reverse=True)
    except:
        print(3)
        return None
    
    # Iterate through the sorted sources and check if they are at least 11 pixels away from the edges
    for star in sorted_sources:
        x, y = int(star['xcentroid']), int(star['ycentroid'])
        if x >= 11 and y >= 11 and x <= 501 and y <= 501:  # 512 - 11 = 501
            # If the star is valid, proceed with the Gaussian fitting
            def gaussian_1d(x, amp, mean, stddev):
                return amp * np.exp(-0.5 * ((x - mean) / stddev) ** 2)
            
            # Extract the cutout around the valid star
            cutout = first_layer[y - size:y + size, x - size:x + size]
            
            try:
                profile = cutout[size, :]
                print([x,y])
            except:
                print(5)
                return None
            
            x_vals = np.arange(len(profile))
            
            if profile.shape[0] < 20:
                print(profile.shape)
                print(5)
                return None
            
            #try:
            popt, _ = curve_fit(gaussian_1d, x_vals, profile, p0=[np.max(profile), size, 1.0])
            amp, mean, stddev = popt
            fwhm_value = 2 * np.sqrt(2 * np.log(2)) * stddev
            return fwhm_value
            #except RuntimeError: 
                #print(6)
                #return None
    
    # If no valid star was found, return None
    print("No valid star found.")
    return None


In [1]:
import numpy as np
from astropy.stats import mad_std
from photutils.detection import DAOStarFinder
from scipy.optimize import curve_fit

def calculate_fwhms(reference_image, image_list, fwhm=3.0, threshold_multiplier=5.0, size=10):
    def gaussian_1d(x, amp, mean, stddev):
        return amp * np.exp(-0.5 * ((x - mean) / stddev) ** 2)

    def calculate_fwhm(image, x, y, size):
        cutout = image[y - size:y + size, x - size:x + size]
        profile = cutout[size, :]
        x_vals = np.arange(len(profile))
        popt, _ = curve_fit(gaussian_1d, x_vals, profile, p0=[np.max(profile), size, 1.0])
        amp, mean, stddev = popt
        return 2 * np.sqrt(2 * np.log(2)) * stddev

    # Detect sources in the reference image
    background_noise = mad_std(reference_image)
    daofind = DAOStarFinder(fwhm=fwhm, threshold=threshold_multiplier * background_noise)
    sources = daofind(reference_image)

    if sources is None:
        print("No stars found in the reference image.")
        return None

    # Sort sources by flux and find the brightest star at least 16 pixels from edges
    sorted_sources = sorted(sources, key=lambda x: x['flux'], reverse=True)
    for star in sorted_sources:
        x, y = int(star['xcentroid']), int(star['ycentroid'])
        if x >= 11 and y >= 11 and x <= 501 and y <= 501:  # Ensure 16-pixel margin
            break
    else:
        print("No valid star found in the reference image.")
        return None

    # Calculate FWHM for the reference image
    fwhms = []
    try:
        ref_fwhm = calculate_fwhm(reference_image, x, y, size)
        fwhms.append(ref_fwhm)
    except RuntimeError:
        print("Fitting failed for the reference image.")
        fwhms.append(None)

    # Calculate FWHM for the corresponding positions in the other images
    for image in image_list:
        try:
            fwhm_value = calculate_fwhm(image, x, y, size)
            fwhms.append(fwhm_value)
        except RuntimeError:
            print("Fitting failed for one of the images.")
            fwhms.append(None)

    return fwhms
